In [155]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.preprocessing import OneHotEncoder

In [156]:
# DROP COLUMNS
notas_red_comp = ['NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5']
nota_red_total = ['NU_NOTA_REDACAO']
situacao_red = ['TP_STATUS_REDACAO']
presenca_na_prova = ['TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT']
codigo_dos_cadernos = ['CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT']

notas_nas_provas = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT']
answer_vector = ['TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT']
gabarito_vector = ['TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC', 'TX_GABARITO_MT']
language_option = ['TP_LINGUA']

columns_to_use = ['NU_INSCRICAO', notas_nas_provas[3], answer_vector[3], gabarito_vector[3]]


## Visualizando nosso dataset de treino

In [158]:
df_train = pd.read_csv("train.csv").dropna()
df_test = pd.read_csv("test.csv")

filtred_train_df = df_train.loc[:14000,list(df_test.columns)]

y_df_train_target = df_train['NU_NOTA_MT']
x_df_train_features = filtred_train_df.drop(columns = ['NU_INSCRICAO'])

x_df_train_features

,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,...,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q006,Q024,Q025,Q026,Q027,Q047
198,23,CE,21,M,1,1,2,0,2,1.0,...,40.0,380.0,C,C,B,A,B,C,F,A
236,12,AC,20,F,3,1,2,0,2,1.0,...,40.0,520.0,A,A,B,B,A,C,D,A
1098,29,BA,19,M,2,1,2,0,2,1.0,...,0.0,440.0,C,C,C,A,A,C,F,A
1361,26,PE,18,M,1,1,2,0,3,1.0,...,80.0,500.0,C,E,H,B,B,C,D,D
1381,31,MG,32,F,2,1,2,0,2,3.0,...,40.0,320.0,C,B,E,A,B,C,B,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12487,23,CE,33,M,3,1,2,0,2,3.0,...,40.0,380.0,H,H,C,A,A,C,D,A
12621,23,CE,19,M,3,1,2,0,2,1.0,...,60.0,520.0,B,E,C,A,A,C,D,A
12672,33,RJ,18,F,2,1,2,0,3,3.0,...,60.0,580.0,E,E,C,A,B,C,C,C
13522,50,MS,23,M,1,1,2,0,2,1.0,...,80.0,560.0,B,B,C,A,A,C,B,A


### Aplicando One hot encoding

In [159]:
def use_one_hot_encoder(df):
    encoder = OneHotEncoder(handle_unknown='ignore').fit(x_df_train_features)
    
    encoder_df = pd.DataFrame(encoder.transform(df).toarray())
    return encoder_df

### Treinando o modelo

In [160]:
linear_regression = LinearRegression()

y_train = y_df_train_target
X_train = use_one_hot_encoder(x_df_train_features)

linear_regression.fit(X_train, y_train)

def fill_faltante_zero(df, alunos_faltantes):
    new_df = df.copy()
    inscri = new_df['NU_INSCRICAO']
    nota = new_df['NU_NOTA_MT']
    for i in range(1, new_df.shape[0]):
        if inscri[i] in alunos_faltantes:
            nota[i] = 0
        nota[i] = round(nota[i], 1)
    return new_df

### testando modelo

In [161]:
df_without_ins = df_test.drop(columns = ['NU_INSCRICAO'])
serie_alunos_faltantes = list(df_test[df_without_ins["TP_PRESENCA_LC"] == 0]['NU_INSCRICAO'])
notes = linear_regression.predict(use_one_hot_encoder(df_without_ins))
answer_df = pd.DataFrame({
    "NU_INSCRICAO": df_test['NU_INSCRICAO'],
    "NU_NOTA_MT": notes
}).reset_index(drop=True)

def_answer = fill_faltante_zero(answer_df, serie_alunos_faltantes)
def_answer['NU_NOTA_MT'] = def_answer['NU_NOTA_MT'].round(1)






def_answer.to_csv("answer.csv", index=False)


def_answer

/tmp/ipykernel_213179/2244533759.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nota[i] = round(nota[i], 1)
/tmp/ipykernel_213179/2244533759.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nota[i] = 0


,NU_INSCRICAO,NU_NOTA_MT
0,73ff9fcc02f0a99919906c942c2e1a1042cdcf98,447.0
1,71a95f9f1b91a82c65ad94abbdf9f54e6066f968,485.2
2,b38a03232f43b11c9d0788abaf060f7366053b6d,470.9
3,70b682d9a3636be23f6120fa9d6b164eb3c6002d,0.0
4,715494628a50142ce8cb17191cfe6d0f3cae0934,391.0
...,...,...
4571,dac0f22429c7f8e3931d0abaf5dfc8e5c772a48b,493.0
4572,a75fa8770257e7c9368d059fe53d9ef431f4bdef,372.6
4573,655fa6306720ff16e825903b5422a46608a77545,509.1
4574,1f4bc3e3d56212d500625bfe8ac78ccff4362293,404.8
